In [108]:
from fastai import *
from fastai.tabular import *
from fastai.tabular.all import *
from fastai.collab import *
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [90]:
path = Path('data/')

df = pd.read_csv(path/'train_dropped_290_without_nans.csv')
test = pd.read_csv(path/'test_dropped_290_without_nans.csv')

y_train=pd.read_csv(path/'y_train.csv')['Culture']
df['Culture']=y_train-1

In [91]:
CAT_NAMES = ['0','1','2']
CONT_NAMES = list((df.drop(['0','1','2','Culture'],axis=1)).columns)
TARGET='Culture'

df[TARGET] = df[TARGET].astype('category')
for i in CAT_NAMES:
    df[i]=df[i].astype('category')
    test[i]=test[i].astype('category')

In [92]:
procs = [Categorify, Normalize]
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

dls = TabularDataLoaders.from_df(df, 
                                 y_names=TARGET, 
                                 cat_names=CAT_NAMES,
                                 cont_names=CONT_NAMES,
                                 procs=procs,
                                 splits=splits)

In [93]:
class_count_df = df.groupby(TARGET).count()
classes=pd.Series(list(class_count_df['0']))
new_weights=[]
for i in classes:
    new_weights.append((classes.sum()) / (2.0 * i))
class_weights=torch.FloatTensor(new_weights)


In [135]:
metrics = F1Score(average='weighted')
loss_func = CrossEntropyLossFlat(weight=class_weights)
learn = tabular_learner(dls,layers=[586,784], loss_func=loss_func,metrics=[metrics])
learn.fit(n_epoch=100)

epoch,train_loss,valid_loss,f1_score,time
0,2.817811,2.494187,0.504023,00:01
1,1.865906,2.174332,0.607697,00:01
2,1.120128,2.003129,0.593831,00:01
3,0.642927,2.227394,0.642376,00:01
4,0.371067,2.436240,0.667444,00:01
5,0.236652,2.497008,0.679366,00:01
6,0.161287,2.623955,0.694230,00:01
7,0.139587,2.524552,0.676094,00:01
8,0.135610,2.783212,0.681139,00:01
9,0.133342,2.899512,0.672532,00:01


KeyboardInterrupt: 

In [136]:
learn.show_results()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,Culture,Culture_pred
0,1.0,779.0,1248.0,-1.123086,-1.100689,-1.078728,-1.058437,-1.045352,-1.037514,-1.033949,-1.031421,-1.030883,-1.031260,-1.043062,-1.055591,-1.067449,-1.077048,-1.084791,-1.089650,-1.091003,-1.088980,-1.085690,-1.077861,-1.079462,-1.079928,-1.074874,-1.069421,-1.062687,-1.055725,-1.051511,-1.045352,-1.044257,-1.040931,-1.037243,-1.034056,-1.028343,-1.020748,-1.011811,-1.001915,-0.990457,-0.977640,-0.966454,-0.955098,-0.950930,-0.947962,-0.945679,-0.944281,-0.943107,-0.942889,-0.943738,-0.950873,-0.960617,-0.971182,-0.983716,-0.998997,-1.013969,-1.028363,-1.041629,-1.053727,-1.064549,-1.074242,-1.083967,-1.092281,-1.101376,-1.108997,-1.115963,-1.124551,-1.133018,-1.140250,-1.150594,-1.161501,-1.171300,-1.180966,-1.189530,-1.197891,-1.205093,-1.212722,-1.220672,-1.228339,-1.237212,-1.254318,-1.282433,-1.311636,-1.332666,-1.336211,-1.317173,-1.282161,-1.255723,-1.230725,-1.206907,-1.183079,-1.162504,-1.145732,-1.128217,-1.109762,-1.091527,-1.072932,-1.054017,-1.036434,-1.017840,-1.000979,-0.982863,-0.964229,-0.943804,-0.922278,-0.898281,-0.903589,-0.926517,-0.937012,-0.943754,-0.947567,-0.948944,-0.949053,-0.948012,-0.946054,-0.944063,-0.942546,-0.941939,-0.942053,-0.942796,-0.944575,-0.947351,-0.951048,-0.955925,-0.962547,-0.970123,-0.978751,-0.988518,-0.998725,-1.009335,-1.019834,-1.011711,-0.981911,-0.951582,-0.928061,-0.903871,-0.879014,-0.853595,-0.827073,-0.799991,-0.771966,-0.742923,-0.713192,-0.683593,-0.654178,-0.623712,-0.591857,-0.549613,-0.490010,-0.417255,-0.335777,-0.250290,-0.165192,-0.085084,-0.038825,0.012141,0.067560,0.126779,0.189016,0.254926,0.328186,0.409391,0.496949,0.585924,0.674108,0.762316,0.862292,0.959805,1.021617,1.014387,0.983705,0.961535,0.938013,0.914520,0.893468,0.877548,0.863165,0.848128,0.832405,0.816731,0.801515,0.780967,0.754413,0.721117,0.682668,0.640273,0.595140,0.549760,0.477239,0.406078,0.335441,0.265983,0.197907,0.129887,0.061910,-0.004942,-0.071843,-0.138413,-0.205319,-0.271392,-0.338410,-0.406319,-0.474423,-0.544721,-0.598636,-0.623288,-0.627098,-0.618695,-0.606247,-0.598541,-0.604339,-0.619960,-0.636732,-0.653425,-0.671583,-0.689772,-0.708655,-0.727592,-0.746037,-0.764646,-0.782201,-0.798641,-0.814130,-0.828599,-0.840567,-0.851092,-0.859150,-0.865877,-0.872601,-0.878976,-0.885069,-0.890294,-0.895985,-0.902180,-0.907938,-0.914505,-0.922645,-0.930455,-0.938074,-0.945294,-0.951224,-0.955597,-0.959083,-0.956547,-0.941494,-0.916689,-0.883463,-0.841742,-0.794219,-0.743289,-0.691586,-0.645168,-0.607775,-0.580735,-0.567241,-0.562184,-0.558236,-0.554986,-0.552601,-0.552665,-0.553132,-0.556117,-0.559009,-0.562576,-0.565776,-0.569333,-0.572097,-0.575466,-0.578028,-0.582468,-0.587086,-0.591322,-0.597936,-0.604259,-0.610815,-0.616628,-0.623021,-0.631473,-0.640124,-0.651369,-0.664541,-0.678914,-0.692545,-0.706510,-0.720419,-0.732986,-0.745179,-0.757854,-0.769894,-0.780984,-0.791453,-0.802314,-0.813538,4.0,4.0
1,4.0,142.0,1249.0,1.459098,1